<a href="https://colab.research.google.com/github/CoryLaidlaw/fastai_tutorial_notes/blob/main/Text_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.text.all import *

# Train a Text Classifier From a Pretrained Model

## Get Data

Using IMDB review data

In [ ]:
path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/imdb.vocab')]

In [ ]:
(path/'train').ls()

(#4) [Path('/root/.fastai/data/imdb/train/pos'),Path('/root/.fastai/data/imdb/train/unsupBow.feat'),Path('/root/.fastai/data/imdb/train/neg'),Path('/root/.fastai/data/imdb/train/labeledBow.feat')]

## Label & Dataloader

Data following ImageNet-style organization, positive and negative reviews in seperate folders (pos and neg). Gather it using TextDataLoaders.from_folder, we will specify the name oif the validation folder, 'test' (default is 'valid').

In [ ]:
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj jim xxmaj carrey is back to much the same role that he played in xxmaj the xxmaj mask , a timid guy who is trying to get ahead in the world but who seems to be plagued with bad luck . xxmaj even when he tries to help a homeless guy from being harassed by a bunch of hoodlums ( and of course they have to be xxmaj mexican , obviously ) , his good will towards his fellow man backfires . xxmaj in that case , it was n't too hard to predict that he was about to have a handful of angry hoodlums , but i like that the movie suggests that things like that should n't be ignored . xxmaj i 'm reminded of the episode of xxmaj michael xxmaj moore 's brilliant xxmaj the xxmaj awful xxmaj truth , when they had a man",pos
3,"xxbos xxmaj i 've rented and watched this movie for the 1st time on xxup dvd without reading any reviews about it . xxmaj so , after 15 minutes of watching xxmaj i 've noticed that something is wrong with this movie ; it 's xxup terrible ! i mean , in the trailers it looked scary and serious ! \n\n i think that xxmaj eli xxmaj roth ( mr . xxmaj director ) thought that if all the characters in this film were stupid , the movie would be funny … ( so stupid , it 's funny … ? xxup wrong ! ) xxmaj he should watch and learn from better horror - comedies such xxunk xxmaj night "" , "" the xxmaj lost xxmaj boys "" and "" the xxmaj return xxmaj of the xxmaj living xxmaj dead "" ! xxmaj those are funny ! \n\n """,neg
4,"xxbos "" a xxmaj damsel in xxmaj distress "" is definitely not one of xxmaj fred xxmaj astaire 's better musicals . xxmaj but even xxmaj astaire 's bad films always had some good moments . \n\n xxmaj in "" damsel , "" xxmaj astaire is xxmaj jerry xxmaj halliday , an xxmaj american musical star who is in xxmaj london on a personal appearance tour . xxmaj he meets xxmaj lady xxmaj alice xxmaj xxunk ( 19 - year - old xxmaj joan xxmaj fontaine ) , a beautiful xxmaj english heiress , who hops into the back of a cab he is taking to escape a mob of admirers . \n\n xxmaj jerry believes that xxmaj alice is being forced into a marriage by her rich aunt . xxmaj he tries to rescue her from her family 's country manor house , but soon discovers that the house",neg
5,"xxbos xxmaj now that xxmaj xxunk ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , 

Library automatically adds some special tokens. ***xxbos*** indicates  the beginning of a text and ***xxmaj*** indicates the next word is capitalized

## Learner

Using AWD LSTM architecture, drop_multi parameter controls the magnitude of all dropouts in that model, and then we are using accuracy to track ho we are doing.

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.468395,0.401299,0.822320,03:18


epoch,train_loss,valid_loss,accuracy,time
0,0.296956,0.258202,0.896480,06:59
1,0.242126,0.219680,0.916360,07:00
2,0.186833,0.193533,0.925320,07:02
3,0.139197,0.191353,0.929120,07:00


In [ ]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxbos xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , is n't much different at all from the previous games ( excluding xxmaj tony xxmaj hawk 3 ) . xxmaj the only thing new that is featured in xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , is the new selection of levels , and tweaked out graphics . xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x offers a new career mode , and that is the 2x career . xxmaj the 2x career is basically xxmaj tony xxmaj hawk 1 career , because there is only about five challenges per level . xxmaj if you missed xxmaj tony xxmaj hawk 1 and 2 , i suggest that you buy xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , but if you have played the first two games , you should still",pos,pos
2,"xxbos xxmaj the xxmaj merchant of xxmaj venice 8 / 10 \n\n ( this review assumes a basic knowledge of the story and so may be thought of as containing spoilers to anyone unfamiliar with the plot . ) \n\n xxmaj as a film version of the famous xxmaj shakespeare play , xxmaj merchant of xxmaj venice does what few adaptations have done since xxmaj polanski 's rather gritty xxmaj macbeth : it takes the spirit of the play and brings it out not just as well as , but better than the play alone could readily achieve . \n\n xxmaj lord of the xxmaj rings xxmaj director , xxmaj peter xxmaj jackson , has argued that in adapting a book or play there are three options : attempt to replicate the original as perfectly as possible , use the original idea as an inspiration but take full license for",pos,pos
3,"xxbos "" empire xxmaj strikes xxmaj back "" director xxmaj irvin xxmaj kershner 's "" never xxmaj say xxmaj never xxmaj again , "" a remake of the 1965 xxmaj james xxmaj bond movie "" thunderball , "" does n't surpasses the xxmaj terence xxmaj young original , but this non - harry xxmaj saltzman & xxmaj albert xxup r. xxmaj broccoli film is well worth watching if you call yourself a 007 aficionado . xxmaj nevertheless , despite its shortage of clever gadgets and the lack of a vibrant musical score , "" never xxmaj say xxmaj never xxmaj again "" rates as an above - average , suspenseful doomsday thriller with top - flight performances by a seasoned cast including xxmaj sean xxmaj connery , xxmaj kim xxmaj basinger , xxmaj klaus xxmaj maria xxmaj brandauer , xxmaj max xxmaj von xxmaj sydow , xxmaj barbara xxmaj carrera",pos,pos
4,"xxbos xxmaj you do n't review xxmaj james xxmaj bond movies , you evaluate them , rate them according to how well they meet expectations . xxmaj there are certain things one has come to expect , even demand of a xxmaj bond film and each individual effort either delivers or it does n't . xxmaj so , here are ten elements that make a xxmaj bond film a xxmaj bond film . xxmaj and even though xxup never xxup say xxup never xxup again is not technically part of the official xxmaj bond filmography , the mere presence of xxmaj sean xxmaj connery returning as 007 makes it something more than merely an honorary member of the series . xxmaj anyway , here 's how it rates on a scale of 1 to 10 : \n\n xxmaj title : xxup never xxup say xxup never xxup again : xxmaj",pos,pos
5,"xxbos xxmaj my qualifications for this review ? i own all the xxmaj alien and xxmaj pr

In [ ]:
learn.predict("I really liked that movie!")

('pos', tensor(1), tensor([0.0082, 0.9918]))

# ULMFit Approach

Previous model used was a language model trained on Wikipedia with the task of predicting the next word after reading all the words before. We took this and fine-tuned it directly to a Movie Review Classifier, but we can do better. Wikipedia English is slightly different then IMDB English. So instead of directly fine-tuningto classifier, we can fine-tune the model to IMDB and then use that as the base for the classifier.

This will give us better results since we are using a model that is good at predicting the next word in a movie review as opposed to a wikipedia article.

## Dataloader for language modeling

Pass valid_pct otherwise it will split data by grandparent folder. We pass 0.1 to get a random 10% of reviews for validation.

In [ ]:
path = untar_data(URLs.IMDB)

In [ ]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)
dls_lm.show_batch()

## Language Model Learner

We are using language_model_learner using AWD_LSTM architecture. We can measure success with accuracy and Perplexity (exponential of the loss) and set the default weight of decay to 0.1. to_fp16 puts learner in mixed precision which helps speeds up training on GPUs with Tensor Cores.

In [ ]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

By default a pretrained learner is in a frozen state, meaning that only the head of the model will train while the body stays frozen.

In [ ]:
learn.fit_one_cycle(1, 1e-2)

We can then unfreeze after loading in the save to fine-tune the modelfine-tune the model after unfreezing

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

Now we can save the model without the final task-specific layer (the layer that is converting activations to probabilities of picking each token in the vocab). This is called the encoder

In [ ]:
learn.save_encoder('finetuned')

## Predictions with LM

Before fine-tuning we can use the model to generate random reviews.

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

## Classifier with LM

### Dataloader

Can gather data similar to before with the difference of loading the exact vocabulary that was used when fine-tuning the LM. We pass this with text_vocab

In [ ]:
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', text_vocab=dls_lm.vocab)

### Learner

Define the text classifier as before and then load the encoder

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn = learn.load_encoder('finetuned')

Next we will train with discrimitive LRs and *gradual unfreezing*. In computer vision we often unfreeze the model all at once, but with NLP classifiers, it is found that freezing a few layers at a time makes a real difference.

In [ ]:
learn.fit_one_cycle(1, 2e-2)

We can pass -2 to freeze_to to freeze all but the last two parameter groups

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

Unfreeze more

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

Unfreeze the whole model

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

# Saving States

Saving states creates a .pth file in learn.path/models. You can then .load this later on.

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')